In [ ]:
# 필요한 라이브러리 로드
import os
import pandas as pd
import xlwings as xw

In [ ]:
# 'adobe_raw' 디렉토리에서 처리할 원본 파일 탐색
# 'New_Weekly_Scoreboard(ThisWeek)_Ken'으로 시작하는 파일 사용
for filename in os.listdir('./adobe_raw/'):
    if filename.startswith('New_Weekly_Scoreboard(ThisWeek)_Ken'):
        scoreboard_file = './adobe_raw/' + filename
        break

In [ ]:
# xlwings로 Excel 파일을 열고 데이터 읽기
# DRM 보호 CSV 파일을 읽기 위해 Excel 앱을 직접 제어하는 xlwings 사용
# try...finally 구문으로 Excel 앱의 확실한 종료 보장
workbook = None
try:
    workbook = xw.Book(scoreboard_file)
    sheet1 = workbook.sheets[0].used_range.value
    df_scoreboard = pd.DataFrame(sheet1)
finally:
    if workbook:
        workbook.app.quit()

In [ ]:
# 원본 데이터 정제
# 보고서의 헤더, 구분선, 불필요한 행들을 제거
df_cleaned = df_scoreboard.iloc[8:]
df_cleaned = df_cleaned[df_cleaned[0] != '##############################################']
df_cleaned = df_cleaned.dropna(subset=[0])

# 컬럼명으로 사용될 행 제거
useless_rows = ['Hour of Day', 'Last Touch Channel']
df_cleaned = df_cleaned[~df_cleaned[0].isin(useless_rows)]
df_cleaned = df_cleaned.reset_index(drop=True).copy()

In [ ]:
# 데이터 테이블별 인덱스 식별
# 하나의 시트 안에 있는 여러 테이블을 분리하기 위해 각 테이블의 제목, 시작, 끝 인덱스를 찾음
title_index = df_cleaned[df_cleaned[0].str.startswith('#', na=False)].index
begin_index = title_index + 1
end_index = title_index[1:].append(pd.Index([len(df_cleaned)]))

In [ ]:
# 동적 변수 생성을 대체하는 딕셔너리 사용
# globals() 대신 딕셔너리를 사용해 생성된 데이터프레임을 관리 (코드 안정성 및 가독성 향상)
dataframes = {}
df_names = []

for ti, bi, ei in zip(title_index, begin_index, end_index):
    # 테이블 제목에서 데이터프레임 이름 생성
    df_name_raw = df_cleaned.iloc[ti][0]
    df_name = "df_" + df_name_raw.replace('# ','').replace(' ','_').replace('(','_').replace(')','').lower()
    df_names.append(df_name)
    
    # 인덱스를 사용해 원본 데이터프레임에서 테이블 분리
    dataframes[df_name] = df_cleaned[bi:ei].copy()

print("생성된 데이터프레임 목록:", df_names)

In [ ]:
# 시간대별 트래픽 데이터(df_hourly_traffic) 전처리
# 1. MultiIndex 컬럼 설정
df_hourly_traffic_col_list = [
    ('Hour',''), ('All', 'All'), ('All', 'Mobile'), ('All', 'PC'),
    ('Campaign', 'All'), ('Campaign', 'Mobile'), ('Campaign', 'PC')
]
df_hourly_traffic_col = pd.MultiIndex.from_tuples(df_hourly_traffic_col_list)
dataframes['df_hourly_traffic'].columns = df_hourly_traffic_col

# 2. 데이터 타입 변환 (오류 방지 로직 추가)
for col in dataframes['df_hourly_traffic'].columns:
    # 'Hour' 컬럼은 24시간 형식으로 변환
    if col == ('Hour',''):
        dataframes['df_hourly_traffic'][col] = (pd.to_numeric(dataframes['df_hourly_traffic'][col], errors='coerce').fillna(0) * 24).round(0).astype(int)
    else:
        dataframes['df_hourly_traffic'][col] = pd.to_numeric(dataframes['df_hourly_traffic'][col], errors='coerce').fillna(0).astype(int)

In [ ]:
# 시간대별 트래픽 데이터(df_hourly_traffic) Melt
# Wide format -> Long format으로 데이터 구조 변경
df_hourly_traffic_melted = pd.melt(
    dataframes['df_hourly_traffic'], 
    id_vars=[('Hour', '')],
    var_name=['PageType', 'DeviceType'], 
    value_name='Visits'
)
df_hourly_traffic_melted.columns = ['Hour', 'PageType', 'DeviceType', 'Visits']
dataframes['df_hourly_traffic_melted'] = df_hourly_traffic_melted # 딕셔너리에 추가

In [ ]:
# 나머지 데이터프레임들 컬럼 정리
dataframes['df_last_touch_channel'] = dataframes['df_last_touch_channel'][[0, 1]]
dataframes['df_last_touch_channel'].columns = ['Last Touch Channel', 'Entries']

dataframes['df_last_touch_channel_with_ptk_publisher'] = dataframes['df_last_touch_channel_with_ptk_publisher'][[0, 1, 2]]
dataframes['df_last_touch_channel_with_ptk_publisher'].columns = ['Last Touch Channel', 'ptk_Publisher', 'Entries']

dataframes['df_inbound_search_keyword_paid'] = dataframes['df_inbound_search_keyword_paid'][[0, 1]]
dataframes['df_inbound_search_keyword_paid'].columns = ['Search Keyword', 'Searches']

dataframes['df_inbound_search_keyword_natural'] = dataframes['df_inbound_search_keyword_natural'][[0, 1]]
dataframes['df_inbound_search_keyword_natural'].columns = ['Search Keyword', 'Searches']

dataframes['df_site_search_keyword'] = dataframes['df_site_search_keyword'][[0, 1]]
dataframes['df_site_search_keyword'].columns = ['Search Keyword', 'Searches']

# 사이트 검색어에서 불필요한 접두어 제거
df_site_search = dataframes['df_site_search_keyword']
df_site_search['Search Keyword'] = df_site_search['Search Keyword'].str.replace('GNB search var:','').str.replace('SRP search var:','').str.replace('search var:','').str.strip()

In [ ]:
# 최종 데이터프레임들을 CSV 파일로 저장
for name, df in dataframes.items():
    # 이름에서 'df_' 접두어 제거하여 파일명 생성
    file_name = name.replace('df_', '') + '.csv'
    output_path = os.path.join('./csv_raw', file_name)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"'{output_path}' 저장 완료")